In [2]:
from collections import defaultdict
import tensorflow as tf
import numpy as np
import tflearn
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import tifffile as tiff
import sys
import sklearn
import time
import PIL
from scipy import misc

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def convert_bmp_to_mask(img):
    w, h, c = img.shape
    mask = np.zeros((w, h, c))
    mask[img == 255] = 1
    mask[img < 255] = 0
    mask = np.concatenate([mask, mask[:, :, 0].reshape(w, h, 1)], axis=2)
    return mask

In [4]:
def scale_percentile(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float32)
    # Get 2nd and 98th percentile
    cp_matrix = np.copy(matrix)
    cp_matrix[cp_matrix < 1] = np.nan
    mins = np.nanpercentile(cp_matrix, 1, axis=0)
    maxs = np.nanpercentile(cp_matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

In [5]:
def calculate_std_mean(img):
    w, h, c = img.shape
    target = np.copy(img)
    target[target == 0] = np.nan
    target = target.reshape(-1, c)
    target_std = np.nanstd(target, axis=0)
    target_mean = np.nanmean(target, axis=0)
    return target_std, target_mean

In [6]:
def scale_percentile_2(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float32)
    # Get 2nd and 98th percentile
    zero_pos = matrix < 1
    matrix[zero_pos] = np.nan
    mins = np.nanpercentile(matrix, 1, axis=0)
    maxs = np.nanpercentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix[zero_pos] = 0
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

In [7]:
FILE_back = '/data/zonghua/satellite/Final/new_data/2017_final.tif'
im_back = tiff.imread(FILE_back)

In [8]:
im_back = np.transpose(im_back, [1, 2, 0])

## 2017

In [9]:
back_1 = cv2.imread("boarders/back_1.bmp")
back_2 = cv2.imread("boarders/back_2.bmp")

In [10]:
where_1 = np.where(back_1 == 255)
right_1 = min(np.max(where_1[1]) + 100, back_1.shape[1])
left_1  = max(np.min(where_1[1]) - 100, 0)
back_1_mask = convert_bmp_to_mask(back_1[:, left_1: right_1])
mask_back_part1 = back_1_mask * im_back[:, left_1: right_1]
back1_target = scale_percentile(mask_back_part1)

In [11]:
where_2 = np.where(back_2 == 255)
right_2 = min(np.max(where_2[1]) + 100, back_2.shape[1])
left_2  = max(np.min(where_2[1]) - 100, 0)
back_2_mask = convert_bmp_to_mask(back_2[:, left_2: right_2])
mask_back_part2 = back_2_mask * im_back[:, left_2: right_2]
back2_target = scale_percentile(mask_back_part2)

In [12]:
back_effective = np.zeros(im_back.shape[:2] + (4,))
back_effective[:, left_1: right_1] += back1_target
back_effective[:, left_2: right_2] += back2_target

In [13]:
cv2.imwrite('boarders/2017.jpg', back_effective[:, :, :3] * 255)

True

In [1]:
np.save('/data/zonghua/satellite/Final/new_data/2017_data', back_effective)

NameError: name 'np' is not defined

# Concat Data

In [32]:
data_2015 = np.load('/data/zonghua/satellite/Final/new_data/2015_data.npy', 'r')
data_2017 = np.load('/data/zonghua/satellite/Final/new_data/2017_data.npy', 'r')
data = np.concatenate([data_2015, data_2017], axis=2)

In [33]:
np.save('/data/zonghua/satellite/Final/new_data/new_data', data)
print("OK1")
data_0 = cv2.flip(data, 0)
np.save("/data/zonghua/satellite/Final/new_data/new_data_0", data_0)
print("OK2")
data_1 = cv2.flip(data, 1)
np.save("/data/zonghua/satellite/Final/new_data/new_data_1", data_1)
print("OK3")
data_2 = cv2.flip(data, -1)
np.save("/data/zonghua/satellite/Final/new_data/new_data_-1", data_2)
print("OK4")

OK1
OK2
OK3
OK4
